## Naive Bayes Clasiffier
### ID: eo9232
### Name: Md Reza
### CSC 5825 - Fall 2021


In [1]:
# Import libraries
import csv
import numpy as np
import random
import math
import pandas as pd
from numpy.random import RandomState

#### Calculate each attribute's mean

In [2]:
def calcMean(eachAttrib):  
    count=0
    num_f=float(len(eachAttrib))
    for j in range(len(eachAttrib)):
        count=count+float(eachAttrib[j])
    return count/num_f

#### Calculate each attribute's standard deviation

In [3]:
def calcStdev(eachAttrib):
    mn = calcMean(eachAttrib)
    s=0.0
    num_fl=len(eachAttrib)
    for k in range(num_fl):
        s=s+pow(float(eachAttrib[k])-mn,2)
    var = s/(float(num_fl-1))
    return math.sqrt(var)

#### Load the dataset

In [4]:
dataframe=csv.reader(open('/u/mreza6/5825/Data/heart.csv','r'))
rng = RandomState()
df_n = list(dataframe)
for i in range(len(df_n)-1):
    df_n[i] = [float(x) for x in df_n[i+1]]

#### Split the 303 instances into training and test sets (8:2) and preprocessing

In [5]:
# Split the data into 80:20 ration
trainSet = int(len(df_n) * 0.80)
xTrain = []
xTest = list(df_n)
for i in range(trainSet):
        index = random.randrange(len(xTest))
        xTrain.append(xTest.pop(index))
        
# Features categorization 
cat={}
for k in range(len(df_n)):
    numClass=float(df_n[k][-1])
    r=df_n[k]
    if(numClass not in cat):
        cat[numClass]=[]
    cat[numClass].append(r)
    
#  Segment the data according to class
dataClass={}
for classNum, r in cat.items():
    classDatarow = [(calcMean(eachAttrib), calcStdev(eachAttrib)) 
                     for eachAttrib in zip(*r)]
    classDatarow = classDatarow[0:13]
    dataClass[classNum] = classDatarow

#### Define the naive bayes classifier with prediction

In [6]:
yPredict = []
for i in range(len(xTest)):
    classProb = {}
    for classNum, row in dataClass.items():
        classProb[classNum] = 1
        for j in range(len(row)):
            meanCalc, stdevCalc = row[j]
            x = float(xTest[i][j])
            if(stdevCalc!=0):
                power = math.exp(-(math.pow(x-meanCalc,2)/(2*math.pow(stdevCalc,2))))
                prob= (1 / (math.sqrt(2*math.pi) *stdevCalc)) * power
            classProb[classNum] *= prob

    classOut, maxProb = -1, -1
    for classNum, prob in classProb.items():
        if classOut == -1 or prob > maxProb:
                maxProb = prob
                classOut = classNum 
        
    yPredict.append(classOut)

#### Calculate NB accuracy

In [7]:
def nbAccuracy(xTest,yPredict):
    count = 0
    for i in range(len(xTest)):
        if xTest[i][-1] == yPredict[i]:
            count += 1
    accuracy=(count/float(len(xTest))) * 100.0
    return accuracy  

#### Define the confusion matrix and calculate precision, recall, & f1-score

In [8]:
def confusionMatrix(xTest,yPredict):
   
    # Count TP
    #===================================
    tp=0
    for i in range(len(xTest)):
        if(xTest[i][-1]==1 and yPredict[i]==1):
            tp=tp+1
    
    # Count FP
    #===================================
    fp=0
    for i in range(len(xTest)):
        if(xTest[i][-1]==0 and yPredict[i]==1):
            fp=fp+1
    
    #Count FN
    #===================================
    fn=0
    for i in range(len(xTest)):
        if(xTest[i][-1]==1 and yPredict[i]==0):
            fn=fn+1
     
    #Count TN
    #===================================
    tn=0
    for i in range(len(xTest)):
        if(xTest[i][-1]==0 and yPredict[i]==0):
            tn=tn+1
    
    # Confusion Matrix
    print("\n\nConfusion Matrix")
    print("\n%15sActual" % "")
    print("%6s %7s %7s" % ("", "1", "0"))
    print("P%6s +--------+--------+" % "")
    print("r%6s | %-6s | %-6s |" % ("1", 'TP='+str(tp), 'FP='+str(fp)))
    print("e%6s +--------+--------+" % "")
    print("d%6s | %-6s | %-6s |" % ("0", 'FN='+str(fn), 'TN='+str(tn)))
    print(".%6s +--------+--------+\n" % "")
    
    # Precision
    precision=(tp)/(tp+fp)
    print("\n\n Precision:", format(precision,".4f"))

    # Recall
    recall=(tp)/(tp+fn)
    print(" Recall   :", format(recall,".4f"))
    
    # F1 Score
    f1score=(2*precision*recall)/(precision+recall)
    print(" F1-Score :",format(f1score,".4f"))

#### Print Accuracy

In [9]:
def printAccuracy(xTest,yPredict):
    acc=nbAccuracy(xTest,yPredict) 
    print(" Accuracy :",format(acc,".2f"),"%")

#### Print confusion matrix, Precision, Recall, F1-Score, & Accuracy

In [10]:
confusionMatrix(xTest,yPredict)
printAccuracy(xTest,yPredict)



Confusion Matrix

               Actual
             1       0
P       +--------+--------+
r     1 | TP=23  | FP=5   |
e       +--------+--------+
d     0 | FN=2   | TN=31  |
.       +--------+--------+



 Precision: 0.8214
 Recall   : 0.9200
 F1-Score : 0.8679
 Accuracy : 88.52 %
